In [ ]:
import os
import cv2
import numpy as np

# Set paths
root_dir = 'D:/Dataset/dana36/views'
output_dir = 'D:/Dataset/dana36/normalized'
os.makedirs(output_dir, exist_ok=True)

# Image size for resizing
image_size = 416


def read_annotations(annotation_path):
    """Reads bounding box annotations from a .txt file and returns a list of 24 rows."""
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    boxes = [list(map(int, line.strip().split(','))) if line.strip() != '0,0,0,0' else [0, 0, 0, 0] for line in lines]

    # Ensure we have exactly 24 rows
    while len(boxes) < 24:
        boxes.append([0, 0, 0, 0])
    return boxes


def resize_with_padding(image, target_size):
    """Resize image with padding to maintain aspect ratio."""
    h, w = image.shape[:2]
    scale = min(target_size / w, target_size / h)
    new_w, new_h = int(w * scale), int(h * scale)
    resized_image = cv2.resize(image, (new_w, new_h))

    # Create a new image with padding
    padded_image = np.full((target_size, target_size, 3), 128, dtype=np.uint8)
    pad_w, pad_h = (target_size - new_w) // 2, (target_size - new_h) // 2
    padded_image[pad_h:pad_h + new_h, pad_w:pad_w + new_w] = resized_image

    return padded_image, scale, pad_w, pad_h


def adjust_bounding_boxes(boxes, scale, pad_w, pad_h):
    """Adjust bounding boxes according to resized image with padding."""
    adjusted_boxes = []
    for (x_min, y_min, x_max, y_max) in boxes:
        if [x_min, y_min, x_max, y_max] == [0, 0, 0, 0]:
            adjusted_boxes.append([0, 0, 0, 0])  # Keep as 0s for non-present objects
        else:
            x_min = int(x_min * scale + pad_w)
            y_min = int(y_min * scale + pad_h)
            x_max = int(x_max * scale + pad_w)
            y_max = int(y_max * scale + pad_h)
            adjusted_boxes.append([x_min, y_min, x_max, y_max])
    return adjusted_boxes


def normalize_image(image):
    """Normalize the image pixel values to [0, 1]."""
    return image.astype(np.float32) / 255.0


# Process images and annotations
for view_num in range(1, 37):  # Loop through views 01 to 36
    view_str = str(view_num).zfill(2)
    view_dir = os.path.join(root_dir, view_str, 'frames')
    annotation_dir = os.path.join(root_dir, view_str, 'annotations')
    output_view_dir = os.path.join(output_dir, view_str)
    os.makedirs(output_view_dir, exist_ok=True)

    for image_file in os.listdir(view_dir):
        if not image_file.endswith('.jpg'):
            continue

        # Read image and annotations
        image_path = os.path.join(view_dir, image_file)
        annotation_path = os.path.join(annotation_dir, image_file.replace('.jpg', '.txt'))
        image = cv2.imread(image_path)
        boxes = read_annotations(annotation_path)

        # Resize and pad image
        resized_image, scale, pad_w, pad_h = resize_with_padding(image, image_size)

        # Adjust bounding boxes
        adjusted_boxes = adjust_bounding_boxes(boxes, scale, pad_w, pad_h)

        # Normalize the image
        normalized_image = normalize_image(resized_image)

        # Save the processed image and annotations
        output_image_path = os.path.join(output_view_dir, image_file)
        output_annotation_path = os.path.join(output_view_dir, image_file.replace('.jpg', '.txt'))
        cv2.imwrite(output_image_path, (normalized_image * 255).astype(np.uint8))  # Save normalized image as uint8
        with open(output_annotation_path, 'w') as f:
            for (x_min, y_min, x_max, y_max) in adjusted_boxes:
                f.write(f'{x_min},{y_min},{x_max},{y_max}\n')

print("Normalization completed!")